In [1]:
# Load the required packages 

import cobra # package for constraint-based metabolic modelling
# See here for documentation: https://cobrapy.readthedocs.io/en/latest/

import requests # for sending HTTP requests; accessing the uniprot website
import xmltodict # to work with the xml requiry from uniprot 

import pandas as pd # for working with dataframes
import numpy as np # for general maths operations

import multiprocessing as mp # for running long tasks in parallel
import time # for checking runtimes

In [2]:
# Load the metabolic model
model = cobra.io.read_sbml_model("./Human-GEM.xml")
# The model was obtained from Robinson et al. 2020
# DOI: 10.1126/scisignal.aaz1482 

# Print model details
rxns = model.reactions
mets = model.metabolites
gens = model.genes
print("The model has {} reactions and {} metabolites and {} genes.".format(len(rxns),len(mets),len(gens)))

The model has 13096 reactions and 8400 metabolites and 3628 genes.


In [3]:
def string_reaction(r):
    """Returns a chemical reaction as an easily readable string"""
    met_dict = {}
    met_dict_name = {}
    for m in r.metabolites:
        coeff = r.get_coefficient(m.id)
        met_dict_name[m.name] = coeff
    substrates_names = dict(filter(lambda x: x[1] < 0.0, met_dict_name.items())) 
    products_names = dict(filter(lambda x: x[1] > 0.0, met_dict_name.items())) 
    subs = [' '.join([str(substrates_names[key]),str(key)]) for key in substrates_names.keys()]
    subs = ' + '.join(subs)
    prods = [' '.join([str(products_names[key]),str(key)]) for key in products_names.keys()]
    prods = ' + '.join(prods)
    if r.lower_bound < 0.0:
        if r.upper_bound == 0.0:
            reaction_string = "{} <-- {}".format(subs,prods)
        else:
            reaction_string = "{} <--> {}".format(subs,prods)
    else: 
        reaction_string = "{} --> {}".format(subs,prods)
    return(reaction_string)

def convert_ENSG_to_uniprot(ENSG_ID):
    """Convert ENSEMBL Id to Uniprot Id"""
    server = "http://www.uniprot.org/uniprot/?query={}&format=xml".format(ENSG_ID)
    r = requests.get(server, headers={"Content-Type": "text/xml"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    response = r.text
    if len(response) > 0:
        # if a response was found for the search item
        info = xmltodict.parse(response)
        try:
            uni_id = info['uniprot']['entry']['accession']
            uni_name = info['uniprot']['entry']['name']
            if isinstance(uni_id, list):
                uni_id = uni_id[0]
            return uni_id,uni_name
        except TypeError:
            uni_id = info['uniprot']['entry'][0]['accession']
            if isinstance(uni_id, list):
                uni_id = uni_id[0]
            uni_name = info['uniprot']['entry'][0]['name']
            return uni_id,uni_name
    else:
        # if no result was found for the search item 
        return "NA","NA"
    
def get_protein_info(r):
    """Get protein information from Uniprot for that reaction."""
    try: # Check for EC number in model annotation
        ec = r.annotation['ec-code']
    except KeyError:# If no EC number is found, annotate with NA
        ec = 'NA'
    # Update the reaction rule with Uniprot IDs
    rxn_rule = r.gene_reaction_rule
    u_names = [] # Enzyme name on uniprot 
    for g in r.genes:
        # One reaction may be catalyzed by different isoforms
        u_id, u_name = convert_ENSG_to_uniprot(g.id)
        u_names.append(u_name)
        rxn_rule = rxn_rule.replace(g.id,u_id)
    unique_u_name = list(set(u_names))
    return [rxn_rule, unique_u_name, ec]

In [4]:
tic = time.time()

# Get protein information for all model reactions using parallel processing
pool = mp.Pool(int(mp.cpu_count()/2)) # Adjust this to wanted # of cores
results = pool.map(get_protein_info,[r for r in model.reactions])
pool.close()

toc = time.time()
print("Runtime: {} second.".format(round(toc-tic,2)))

Runtime: 10250.82 second.


In [5]:
# Gather all of the reaction/protein info into a dataframe 
reaction_ids = [r.id for r in model.reactions]
reaction_str = [string_reaction(r) for r in model.reactions]
reaction_rules = [r[0] for r in results]
uniprot_names = [r[1] for r in results]
reaction_ecs = [r[2] for r in results]

# Save the dataframe as a csv file 
data = {"ReactionID":reaction_ids, "Reaction":reaction_str, "UniprotID":uniprot_names,
        "EC":reaction_ecs,"ReactionRule":reaction_rules}
df = pd.DataFrame(data)
df.to_csv('model_proteins.csv',index=False,header=True)

In [6]:
# Annotate the model and save the above results 
# Can now use the annotated model to access protein information directly
for i,r in enumerate(model.reactions):
    r.name = reaction_str[i]
    r.annotation['ec-code'] = reaction_ecs[i]
    r.annotation['uniprot.name'] = uniprot_names[i]
    r.notes['protein_reaction_rule'] = reaction_rules[i]
    
# Save the model to file
cobra.io.write_sbml_model(model, "Human-GEM-annotated.xml")
# Use the update model for further analyses

In [7]:
# CHECK: reload the model and view added info
model = cobra.io.read_sbml_model("./Human-GEM-annotated.xml")
for r in model.reactions[0:3]:
    print(r.id)
    print(r.name)
    print(r.annotation)
    print(r.notes)
    print(" ")

HMR_3905
-1.0 ethanol + -1.0 NAD+ --> 1.0 acetaldehyde + 1.0 H+ + 1.0 NADH
{'sbo': 'SBO:0000176', 'ec-code': ['1.1.1.1', '1.1.1.71'], 'kegg.reaction': 'R00754', 'bigg.reaction': 'ALCD2x', 'metanetx.reaction': 'MNXR95725', 'uniprot.name': ['HOT_HUMAN', 'ADH1G_HUMAN', 'ADHX_HUMAN', 'PTGR3_HUMAN', 'ADH1B_HUMAN', 'ADH1A_HUMAN', 'ADH6_HUMAN', 'ADH7_HUMAN', 'ADH4_HUMAN']}
{'Confidence Level': '0', 'AUTHORS': 'PMID:10868354;PMID:12491384;PMID:12818203;PMID:14674758;PMID:15289102;PMID:15299346;PMID:15327949;PMID:15682493;PMID:15713978', 'protein_reaction_rule': 'Q8IWW8 or P28332 or Q8N4Q0 or P07327 or P40394 or P00325 or P11766 or P08319 or P00326'}
 
HMR_3907
-1.0 ethanol + -1.0 NADP+ --> 1.0 acetaldehyde + 1.0 H+ + 1.0 NADPH
{'sbo': 'SBO:0000176', 'ec-code': '1.1.1.2', 'kegg.reaction': 'R00746', 'bigg.reaction': 'ALCD2y', 'metanetx.reaction': 'MNXR95726', 'uniprot.name': 'AK1A1_HUMAN'}
{'Confidence Level': '0', 'protein_reaction_rule': 'P14550'}
 
HMR_4097
-1.0 acetate + -1.0 ATP + -1.0 CoA 